# MODELLING & EVALUATION

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('cleaned_cars.csv')

In [3]:
df

,Brand,Model,Year,Kilometers_Driven,Mileage,Price,Fuel_Type_Petrol,Owner_Type_Second,log_Engine,log_Power,boxcox_Seats
0,9,15,2018,50000,15,800000,1,0,7.311886,4.682131,0.397292
1,3,14,2019,40000,17,1000000,1,1,7.375882,4.941642,0.397292
2,2,30,2017,20000,10,2500000,1,0,8.507345,5.978886,0.391701
3,6,42,2020,30000,23,600000,0,0,7.129298,4.304065,0.397292
4,4,41,2016,60000,18,850000,0,1,7.600402,5.267858,0.397292
...,...,...,...,...,...,...,...,...,...,...,...
95,7,11,2019,22000,16,2900000,0,0,7.575585,5.252273,0.397292
96,9,29,2017,38000,13,1400000,0,1,7.921173,5.141664,0.401583
97,2,18,2018,26000,18,750000,1,0,7.311218,4.795791,0.397292
98,4,50,2019,24000,17,850000,1,1,7.311218,4.727388,0.397292


In [4]:
x=df.drop(columns=['Price'])
y=df['Price']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
cv=[]
train=[]
for i in range(1,100):
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=i)
    model=LinearRegression()
    model.fit(x_train,y_train)
    train_r2=model.score(x_train,y_train)
    CV=cross_val_score(model,x_train,y_train,cv=5,scoring='r2').mean()
    train.append(train_r2)
    cv.append(CV)    

In [6]:
df1=pd.DataFrame({'Train':train,'CV':cv})

In [7]:
df2=df1[(df1['Train']-df1['CV'])<=0.5]

In [8]:
df2[df2['CV']==df2['CV'].max()]

,Train,CV
69,0.891342,0.83998


In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=69)

# LINEAR REGRESSION

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=LinearRegression()
model.fit(x_train,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train)
ypred_test=model.predict(x_test)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)



coefficeint: [ 4.66637018e+03 -6.55352749e+03 -1.26840857e+04 -2.10606019e+01
  1.66944986e+03 -3.30862172e+05 -3.44145797e+04 -5.09434674e+05
  2.34304804e+06  6.37143156e+07]
intercept: -5070345.976959741
mse of train: 141949300888.47427
mse of test: 141623418327.93018
RMSE: 376328.8699102557
R2_score of Train Data: 0.858798131346729
R2_score of Test Data: 0.8473669704600676
cross_val_score : 0.7484558805448249


# POLYNOMIAL REGRESSION

In [11]:
from sklearn.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(degree=2)
x_train_poly=pf.fit_transform(x_train)
x_test_poly=pf.transform(x_test)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=LinearRegression()
model.fit(x_train_poly,y_train)
#print('coefficeint:',model.coef_)
#print('intercept:',model.intercept_)

ypred_train=model.predict(x_train_poly)
ypred_test=model.predict(x_test_poly)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train_poly,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)



mse of train: 6284839.957449859
mse of test: 1544613020695.9255
RMSE: 1242824.613811589
R2_score of Train Data: 0.9999937482527873
R2_score of Test Data: -0.6646891283879597
cross_val_score : -19.860862183442894


# LASSO

In [13]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
estimator=Lasso()
param_grid={'alpha':list(range(1,10))}
gd=GridSearchCV(estimator,param_grid,cv=5)
gd.fit(x_train,y_train)
gd.best_params_


{'alpha': 1}

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=Lasso(alpha=1)
model.fit(x_train,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train)
ypred_test=model.predict(x_test)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)



coefficeint: [ 4.67393743e+03 -6.56278032e+03 -1.25984487e+04 -2.10357089e+01
  1.61302523e+03 -3.31259828e+05 -3.41052123e+04 -5.08201830e+05
  2.34150156e+06  6.33187946e+07]
intercept: -5086911.932024872
mse of train: 141949699225.62164
mse of test: 141699936462.20123
RMSE: 376430.5200992625
R2_score of Train Data: 0.8587977351070208
R2_score of Test Data: 0.8472845039104925
cross_val_score : 0.748280665660129


# RIDGE

In [15]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
estimator=Ridge()
param_grid={'alpha':list(range(1,20))}
gd=GridSearchCV(estimator,param_grid,cv=5)
gd.fit(x_train,y_train)
gd.best_params_

{'alpha': 1}

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=Ridge(alpha=1)
model.fit(x_train,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train)
ypred_test=model.predict(x_test)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)


coefficeint: [-7.29143388e+03 -8.14666585e+03 -2.15148434e+04 -1.93829740e+01
 -1.82067186e+04 -3.33275144e+05  2.40625015e+04  1.98864031e+05
  1.57224040e+06  8.02469748e+03]
intercept: 36991148.72487668
mse of train: 161691802911.66425
mse of test: 181350043379.93964
RMSE: 425852.13793045544
R2_score of Train Data: 0.8391595832163962
R2_score of Test Data: 0.8045520518069612
cross_val_score : 0.7091528819443147


# ELASTIC NET

In [17]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
estimator=ElasticNet()
param_grid={'alpha':list(range(1,50)),'l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}
gd=GridSearchCV(estimator,param_grid,cv=5)
gd.fit(x_train,y_train)
gd.best_params_

{'alpha': 1, 'l1_ratio': 1.0}

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=ElasticNet(alpha=1,l1_ratio=1.0)
model.fit(x_train,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train)
ypred_test=model.predict(x_test)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)


coefficeint: [ 4.67393743e+03 -6.56278032e+03 -1.25984487e+04 -2.10357089e+01
  1.61302523e+03 -3.31259828e+05 -3.41052123e+04 -5.08201830e+05
  2.34150156e+06  6.33187946e+07]
intercept: -5086911.932024872
mse of train: 141949699225.62164
mse of test: 141699936462.20123
RMSE: 376430.5200992625
R2_score of Train Data: 0.8587977351070208
R2_score of Test Data: 0.8472845039104925
cross_val_score : 0.748280665660129


# APPLYING PCA

In [19]:
from sklearn.decomposition import PCA
pca=PCA(n_components=0.95)
x_train_pca=pca.fit_transform(x_train)
x_test_pca=pca.transform(x_test)

# LINEAR

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=LinearRegression()
model.fit(x_train_pca,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train_pca)
ypred_test=model.predict(x_test_pca)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train_pca,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)



coefficeint: [-4.55220153]
intercept: 1560625.0
mse of train: 1003417710564.0289
mse of test: 927305122227.9336
RMSE: 962966.8333997458
R2_score of Train Data: 0.0018657726060552493
R2_score of Test Data: 0.0006074434256637673
cross_val_score : -0.1290063068044637


# LASSO

In [21]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
estimator=Lasso()
param_grid={'alpha':list(range(1,10))}
gd=GridSearchCV(estimator,param_grid,cv=5)
gd.fit(x_train_pca,y_train)
gd.best_params_

{'alpha': 9}

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model_lasso=Lasso(alpha=9)
model_lasso.fit(x_train_pca,y_train)
print('coefficeint:',model_lasso.coef_)
print('intercept:',model_lasso.intercept_)

ypred_train=model_lasso.predict(x_train_pca)
ypred_test=model_lasso.predict(x_test_pca)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model_lasso,x_train_pca,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)



coefficeint: [-4.55220143]
intercept: 1560625.0
mse of train: 1003417710564.0287
mse of test: 927305122315.3738
RMSE: 962966.8334451471
R2_score of Train Data: 0.0018657726060553603
R2_score of Test Data: 0.0006074433314261496
cross_val_score : -0.12900630634253457


# RIDGE

In [23]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
estimator=Ridge()
param_grid={'alpha':list(range(1,500))}
gd=GridSearchCV(estimator,param_grid,cv=5)
gd.fit(x_train_pca,y_train)
gd.best_params_

{'alpha': 499}

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=Ridge(alpha=499)
model.fit(x_train_pca,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train_pca)
ypred_test=model.predict(x_test_pca)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train_pca,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)


coefficeint: [-4.55220122]
intercept: 1560625.0
mse of train: 1003417710564.0289
mse of test: 927305122503.8018
RMSE: 962966.8335429843
R2_score of Train Data: 0.0018657726060552493
R2_score of Test Data: 0.0006074431283500381
cross_val_score : -0.12900630165967913


# ELASTIC NET

In [25]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
estimator=ElasticNet()
param_grid={'alpha':list(range(1,50)),'l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}
gd=GridSearchCV(estimator,param_grid,cv=5)
gd.fit(x_train_pca,y_train)
gd.best_params_

{'alpha': 49, 'l1_ratio': 0.1}

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
model=ElasticNet(alpha=49,l1_ratio=0.1)
model.fit(x_train_pca,y_train)
print('coefficeint:',model.coef_)
print('intercept:',model.intercept_)

ypred_train=model.predict(x_train_pca)
ypred_test=model.predict(x_test_pca)

print("mse of train:",mean_squared_error(y_train,ypred_train))
print("mse of test:",mean_squared_error(y_test,ypred_test))
print("RMSE:",(mean_squared_error(y_test,ypred_test))**(1/2))

print("R2_score of Train Data:",r2_score(y_train,ypred_train))
print("R2_score of Test Data:",r2_score(y_test,ypred_test))

cv=cross_val_score(model,x_train_pca,y_train,cv=5,scoring='r2').mean()
print("cross_val_score :",cv)

coefficeint: [-4.55219926]
intercept: 1560625.0
mse of train: 1003417710564.0293
mse of test: 927305124225.9681
RMSE: 962966.8344371825
R2_score of Train Data: 0.0018657726060548052
R2_score of Test Data: 0.0006074412723047207
cross_val_score : -0.12900627745350426
